In [1]:
import tensorflow as tf 
from PIL import Image #python image library
import numpy as np
import os 

def write_record(root,filename):
    writer=tf.python_io.TFRecordWriter(filename)
    for i in range(10):
        class_path = root+"Sample0{:02}/".format(i+1)
        for img_name in os.listdir(class_path):
            if img_name == '.DS_Store':
                continue
            img_path = class_path+img_name
            img = Image.open(img_path).convert("L") #轉成grayscale
            img_raw = img.tobytes()
            labels  = [0]*10
            labels[i] = 1
            #Example 裡面裝features
            #features裡面裝feature
            example=tf.train.Example(features=tf.train.Features(
              feature={
                "label":tf.train.Feature(int64_list=tf.train.Int64List(value=labels)),
                'img_raw':tf.train.Feature(bytes_list=tf.train.BytesList(
                                                                    value=[img_raw]))
            }))
            writer.write(example.SerializeToString())
    writer.close()        


In [2]:
def read_record(filename,batch_size,num_epochs):
    #建立檔名佇列
    filename_queue = tf.train.string_input_producer([filename],num_epochs=num_epochs)
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    #將「tf.train.Example」檔案解析為「tf.train.Features」
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label':tf.FixedLenFeature([10],tf.int64),
                                           'img_raw':tf.FixedLenFeature([],tf.string)
                                       })
    #將tobyte()改寫回來成tf.uint8
    img = tf.decode_raw(features['img_raw'],tf.uint8)
    img = tf.reshape(img,(128,128,1))
    img = tf.cast(img,tf.float32)/255-0.5
    label = tf.cast(features['label'],tf.float32)
    batch_xs,batch_ys = tf.train.shuffle_batch([img,label],batch_size=batch_size,
                        capacity=2000,min_after_dequeue=1000)
    #capacity是隊列的長度
    #min_after_dequeue是 隊列至少剩下min_after_dequeue個數據
    #min_after_dequeue越大，數據越亂
    return batch_xs,batch_ys  

In [21]:

#建立權重weight張量
def weight(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')
#建立偏差(bias)張量
def bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape),name='b')
#定義conv2d函數，用於進行卷積運算
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
#建立max_pool_2*2函數
def max_pool_2_2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')



#def build_graph(withOpt = True):
def build_graph(withOpt = True):
    #with tf.name_scope('Input_Layer'):
    x = tf.placeholder("float32",shape=[None,128,128,1],name="x")
    #with tf.name_scope('C1_Conv'):
    W1 = weight([5,5,1,16])
    b1 = bias([16])
    Conv1 = conv2d(x,W1) + b1
    C1_Conv = tf.nn.relu(Conv1)
    #with tf.name_scope('C1_Pool'):
    C1_Pool = max_pool_2_2(C1_Conv)
    #with tf.name_scope('C2_Conv'):
    W2 = weight([5,5,16,36])
    b2 = bias([36])
    Conv2 = conv2d(C1_Pool,W2)+b2
    C2_Conv = tf.nn.relu(Conv2)
    #with tf.name_scope('C2_Pool'):
    C2_Pool = max_pool_2_2(C2_Conv)
    #with tf.name_scope('D_Flat'):
    D_Flat = tf.reshape(C2_Pool,[-1,32*32*36])

    #with tf.name_scope('D_Hidden_Layer'):
    W3 = weight([32*32*36,128])
    b3 = bias([128])
    D_Hidden = tf.nn.relu(tf.matmul(D_Flat,W3)+b3)
    D_Hidden_Dropout = tf.nn.dropout(D_Hidden,keep_prob=0.8)
    #with tf.name_scope('Output_Layer'):
    W4 = weight([128,10])
    b4 = bias([10])
    y_predict = tf.nn.softmax(tf.matmul(D_Hidden_Dropout,W4)+b4)

    #with tf.name_scope("optimizer"):
    y_label = tf.placeholder("float32",shape=[None,10],name="y_label")
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                                   (logits=y_predict,labels=y_label))
        #optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)
    #with tf.name_scope("evaluate_model"):
    correct_prediction = tf.equal(tf.argmax(y_predict,1),tf.argmax(y_label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float32"))

    if withOpt:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)
        #train_step = tf.train.AdamOptimizer(1e-5).minimize(loss)
        return x, y_label, loss_function, optimizer
    else:
        return x, y_label, loss_function, y_predict





# #建立權重weight張量
# def weight(shape):
#     return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')
# #建立偏差(bias)張量
# def bias(shape):
#     return tf.Variable(tf.constant(0.1,shape=shape),name='b')
# #定義conv2d函數，用於進行卷積運算
# def conv2d(x,W):
#     return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
# #建立max_pool_2*2函數
# def max_pool_2_2(x):
#     return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# with tf.name_scope('Input_Layer'):
#     x = tf.placeholder("float32",shape=[None,128,128,1],name="x")
# with tf.name_scope('C1_Conv'):
#     W1 = weight([5,5,1,16])
#     b1 = bias([16])
#     Conv1 = conv2d(x,W1) + b1
#     C1_Conv = tf.nn.relu(Conv1)
# with tf.name_scope('C1_Pool'):
#     C1_Pool = max_pool_2_2(C1_Conv)
# with tf.name_scope('C2_Conv'):
#     W2 = weight([5,5,16,36])
#     b2 = bias([36])
#     Conv2 = conv2d(C1_Pool,W2)+b2
#     C2_Conv = tf.nn.relu(Conv2)
# with tf.name_scope('C2_Pool'):
#     C2_Pool = max_pool_2_2(C2_Conv)
# with tf.name_scope('D_Flat'):
#     D_Flat = tf.reshape(C2_Pool,[-1,32*32*36])
    
# with tf.name_scope('D_Hidden_Layer'):
#     W3 = weight([32*32*36,128])
#     b3 = bias([128])
#     D_Hidden = tf.nn.relu(tf.matmul(D_Flat,W3)+b3)
#     D_Hidden_Dropout = tf.nn.dropout(D_Hidden,keep_prob=0.8)
# with tf.name_scope('Output_Layer'):
#     W4 = weight([128,10])
#     b4 = bias([10])
#     y_predict = tf.nn.softmax(tf.matmul(D_Hidden_Dropout,W4)+b4)
# with tf.name_scope("optimizer"):
#     y_label = tf.placeholder("float32",shape=[None,10],name="y_label")
#     loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
#                                    (logits=y_predict,labels=y_label))
#     optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)
    
# with tf.name_scope("evaluate_model"):
#     correct_prediction = tf.equal(tf.argmax(y_predict,1),tf.argmax(y_label,1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float32"))

In [18]:
def train():
    b_img, b_mask = read_record("train.tfrecords", 65, 3)

    x, y, loss, train_step = build_graph()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        coord  = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        try:
            i = 0
            while not coord.should_stop():
                i+=1 
                img, mask_ = sess.run([b_img, b_mask])
                sess.run(train_step, feed_dict={x: img, y: mask_})
                if i % 50 == 0:
                    print("loss: ", sess.run(loss, feed_dict={x: img, y: mask_}))
                
        except tf.errors.OutOfRangeError:
            print("Done training")
            saver.save(sess, "./example.ckpt")
        finally:
            coord.request_stop()
            coord.join(threads)

In [5]:
# def train():
#     b_img,b_lab = read_record("train.tfrecords",10,20)
#     #build_graph()
#     saver = tf.train.Saver()
#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
#         sess.run(tf.local_variables_initializer())
#         coord = tf.train.Coordinator()
#         threads = tf.train.start_queue_runners(coord=coord)
#         try:
#             i=0
#             while not coord.should_stop():
#                 i+=1
#                 img,lab = sess.run([b_img,b_lab])
#                 sess.run(optimizer,feed_dict={x:img,y_label:lab})
#                 if i%50 == 0:
#                     print("test")
#                     print("loss: ",sess.run(loss_function,feed_dict={x:img,y_label:lab}))
#                     print("pred: ",np.argmax(sess.run(y_predict,feed_dict={x:img}),axis=1))
#                     print("ans: " ,np.argmax(lab,axis=1))
#         except tf.errors.OutOfRangeError:
#             print("Done training")
#             saver.save(sess,"./example.ckpt")
#         finally:
#             coord.request_stop()
#             coord.join(threads)


In [17]:
def test():
    b_img, b_mask = read_record("test.tfrecords", 10, 20)

    x, y, loss, out = build_graph(withOpt=False)

    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord = coord)
        saver.restore(sess, "./example.ckpt")
        try:
            i = 0
            while not coord.should_stop():
                i+=1 
                img, mask_ = sess.run([b_img, b_mask])
                pred = sess.run(out, feed_dict={x: img, y: mask_})
                pred = np.argmax(pred, axis=3)
                mask_ = np.argmax(mask_, axis=3)
#                 plt.imshow(((img[0] + 0.5) * 255).astype(np.uint8))
#                 plt.show()
#                 plt.savefig("image.jpg")
#                 plt.imshow(pred[0])
#                 plt.show()
#                 plt.savefig("pred.jpg")
#                 plt.imshow(mask_[0])
#                 plt.show()
#                 plt.savefig("mask_.jpg")
                raise Exception
                
        except tf.errors.OutOfRangeError:
            print("Done testing")
        finally:
            coord.request_stop()
            coord.join(threads)

In [22]:
root = "training/"
filename="train.tfrecords"
write_record(root,filename)
root = "validation/"
filename="test.tfrecords"
write_record(root,filename)
train()
test()



Done training


INFO:tensorflow:Restoring parameters from ./example.ckpt
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


NotFoundError: Key W_4 not found in checkpoint
	 [[Node: save_8/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_8/Const_0_0, save_8/RestoreV2/tensor_names, save_8/RestoreV2/shape_and_slices)]]

Caused by op 'save_8/RestoreV2', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages

In [ ]:
# x = tf.placeholder(tf.float32,[None,128,128,1])
# y = tf.placeholder(tf.float32,[None,10])
# #1st convolutional layer
# w1 = tf.Variable(tf.truncated_normal([3,3,1,16],stddev=0.01))
# b1 = tf.Variable(tf.constant(0.1,shape=[16]))
# conv1 = tf.nn.conv2d(x,w1,strides=[1,1,1,1],padding='SAME')
# act1  = tf.nn.tanh(conv1+b1)
# out1  = tf.nn.max_pool(act1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# #2nd convolutional layer
# w2 = tf.Variable(tf.truncated_normal([3,3,16,64],stddev=0.01))
# b2 = tf.Variable(tf.constant(0.1,shape=[64]))
# conv2 = tf.nn.conv2d(out1,w2,strides=[1,1,1,1],padding='SAME')
# act2  = tf.nn.tanh(conv2+b2)
# out2  = tf.nn.max_pool(act2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# flatten = tf.reshape(out2,[-1,32*32*64])
# w3 = tf.Variable(tf.truncated_normal([32*32*64,10],stddev=0.01))
# b3 = tf.Variable(tf.constant(0.1,shape=[10]))
# pre_out = tf.matmul(flatten,w3)+b3
# #loss layer
# loss = tf.losses.softmax_cross_entropy(y,pre_out)
# train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(loss)
